In [141]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import datetime
from pandas import to_datetime
import seaborn as sn
import matplotlib.pyplot as plt
from collections import defaultdict
from scipy.stats.stats import pearsonr, iqr
import pickle
import datetime
datetime.datetime.strptime

<function datetime.strptime>

In [142]:
customer_sequence = pd.read_csv("../../preprocessed.csv", index_col=0)
customer_sequence

,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta
0,536365,2010-01-12 08:26:00,2.55,17850.0,United Kingdom,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6
1,536365,2010-01-12 08:26:00,3.39,17850.0,United Kingdom,71053,WHITE METAL LANTERN,6
2,536365,2010-01-12 08:26:00,2.75,17850.0,United Kingdom,84406B,CREAM CUPID HEARTS COAT HANGER,8
3,536365,2010-01-12 08:26:00,3.39,17850.0,United Kingdom,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6
4,536365,2010-01-12 08:26:00,3.39,17850.0,United Kingdom,84029E,RED WOOLLY HOTTIE WHITE HEART.,6
...,...,...,...,...,...,...,...,...
541904,581587,2011-09-12 12:50:00,0.85,12680.0,France,22613,PACK OF 20 SPACEBOY NAPKINS,12
541905,581587,2011-09-12 12:50:00,2.10,12680.0,France,22899,CHILDREN'S APRON DOLLY GIRL,6
541906,581587,2011-09-12 12:50:00,4.15,12680.0,France,23254,CHILDRENS CUTLERY DOLLY GIRL,4
541907,581587,2011-09-12 12:50:00,4.15,12680.0,France,23255,CHILDRENS CUTLERY CIRCUS PARADE,4


In [143]:
customer_sequence["BasketDate"] = pd.to_datetime(customer_sequence["BasketDate"])
customer_sequence_2011 = customer_sequence[customer_sequence["BasketDate"].dt.year == 2011]
customer_sequence_Nov = customer_sequence_2011[customer_sequence_2011["BasketDate"].dt.month.between(11, 11)]
customer_sequence_Nov

,BasketID,BasketDate,Sale,CustomerID,CustomerCountry,ProdID,ProdDescr,Qta
52127,540688,2011-11-01 07:57:00,18.00,12736.0,France,POST,POSTAGE,5
52135,540690,2011-11-01 08:45:00,1.95,13473.0,United Kingdom,22222,CAKE PLATE LOVEBIRD WHITE,12
52136,540690,2011-11-01 08:45:00,9.95,13473.0,United Kingdom,22782,SET 3 WICKER STORAGE BASKETS,4
52140,540690,2011-11-01 08:45:00,1.95,13473.0,United Kingdom,84766,SILVER ROCOCO CANDLE STICK,12
52144,540690,2011-11-01 08:45:00,3.75,13473.0,United Kingdom,15060B,FAIRY CAKE DESIGN UMBRELLA,4
...,...,...,...,...,...,...,...,...
516361,579885,2011-11-30 17:37:00,1.45,15444.0,United Kingdom,23035,DRAWER KNOB CERAMIC IVORY,6
516363,579885,2011-11-30 17:37:00,3.75,15444.0,United Kingdom,84882,GREEN WIRE STANDING CANDLE HOLDER,2
516364,579885,2011-11-30 17:37:00,1.25,15444.0,United Kingdom,85034C,3 ROSE MORRIS BOXED CANDLES,4
516365,579885,2011-11-30 17:37:00,5.95,15444.0,United Kingdom,21742,LARGE ROUND WICKER PLATTER,2


In [144]:
customer_sequence_Nov=customer_sequence_Nov.groupby(["CustomerID",'BasketID','BasketDate'])

In [145]:
customer_sequence_Nov=customer_sequence_Nov["ProdDescr"].apply(list).to_frame()

In [146]:
#customer_sequence=customer_sequence.groupby(["CustomerID"]).apply(list)
customer_sequence_Nov=customer_sequence_Nov.sort_values(by=['BasketDate'])
customer_sequence_Nov

,,,ProdDescr
CustomerID,BasketID,BasketDate,
12736.0,540688,2011-11-01 07:57:00,[POSTAGE]
13473.0,540690,2011-11-01 08:45:00,"[CAKE PLATE LOVEBIRD WHITE, SET 3 WICKER STORA..."
17450.0,540691,2011-11-01 08:50:00,"[WHITE SPOT BLUE CERAMIC DRAWER KNOB, WHITE SP..."
14320.0,540692,2011-11-01 09:12:00,"[VINTAGE HEADS AND TAILS CARD GAME , WOODEN SC..."
16670.0,540697,2011-11-01 09:20:00,"[GREY HEART HOT WATER BOTTLE, SCOTTIE DOG HOT ..."
...,...,...,...
15544.0,579868,2011-11-30 16:43:00,"[ROLL WRAP 50'S CHRISTMAS, ROLL WRAP 50'S RED ..."
12437.0,579870,2011-11-30 16:47:00,"[RED TOADSTOOL LED NIGHT LIGHT, SAVE THE PLANE..."
14085.0,579872,2011-11-30 16:54:00,"[SET OF 5 MINI GROCERY MAGNETS, FELTCRAFT PRIN..."


In [147]:
customer_sequence_Nov=customer_sequence_Nov.groupby(["CustomerID"])["ProdDescr"].apply(list)

In [148]:
customer_sequence_Nov

CustomerID
12349.0    [[PARISIENNE CURIO CABINET, SWEETHEART WALL TI...
12356.0    [[REGENCY CAKESTAND 3 TIER, RED RETROSPOT CAKE...
12362.0    [[SPACEBOY CHILDRENS CUP,  DOLLY GIRL BEAKER, ...
12363.0    [[DOORMAT RED RETROSPOT, DOORMAT WELCOME SUNRI...
12371.0    [[ASSORTED BOTTLE TOP  MAGNETS , FRIDGE MAGNET...
                                 ...                        
18241.0    [[PLEASE ONE PERSON METAL SIGN, DOORMAT 3 SMIL...
18249.0    [[FAWN BLUE HOT WATER BOTTLE, RABBIT NIGHT LIG...
18255.0    [[WHITE HANGING HEART T-LIGHT HOLDER, SILVER H...
18259.0                     [[SCOTTIE DOG HOT WATER BOTTLE]]
18283.0    [[ROLL WRAP VINTAGE CHRISTMAS, PLASTERS IN TIN...
Name: ProdDescr, Length: 1378, dtype: object

In [149]:
customer_sequence_Nov.iloc[1]

[['REGENCY CAKESTAND 3 TIER', 'RED RETROSPOT CAKE STAND']]

In [150]:
customer_sequence_Nov=customer_sequence_Nov.values.tolist()

In [151]:
len(customer_sequence_Nov)

1378

In [152]:
customer_sequence_Nov[2]

[['SPACEBOY CHILDRENS CUP',
  ' DOLLY GIRL BEAKER',
  'SPACEBOY BEAKER',
  'BUNDLE OF 3 SCHOOL EXERCISE BOOKS  ',
  'MAGNETS PACK OF 4 RETRO PHOTO',
  '12 PENCILS TALL TUBE RED RETROSPOT',
  '12 PENCILS TALL TUBE WOODLAND',
  'WOODLAND  HEIGHT CHART STICKERS ',
  'PLASTERS IN TIN WOODLAND ANIMALS',
  'PACK OF 12 COLOURED PENCILS',
  'TEA FOR ONE POLKADOT',
  'PLASTERS IN TIN STRONGMAN',
  'ROUND SNACK BOXES SET OF4 WOODLAND ',
  'LUNCH BAG CARS BLUE',
  'LUNCH BAG WOODLAND',
  'ROUND SNACK BOXES SET OF 4 FRUITS ',
  'SPACEBOY LUNCH BOX ',
  'DOLLY GIRL LUNCH BOX',
  'LUNCH BOX WITH CUTLERY RETROSPOT ',
  'LUNCH BAG APPLE DESIGN',
  'LUNCH BAG VINTAGE DOILY ',
  'LUNCH BAG DOLLY GIRL DESIGN',
  'LUNCH BOX I LOVE LONDON',
  'DOLLY GIRL CHILDRENS CUP',
  'SPACEBOY CHILDRENS BOWL',
  'POSTAGE'],
 ['LUNCH BOX WITH CUTLERY RETROSPOT ',
  'ROUND SNACK BOXES SET OF 4 FRUITS ',
  'LUNCH BAG SUKI DESIGN ',
  'SPACEBOY LUNCH BOX ',
  'DINOSAUR LUNCH BOX WITH CUTLERY',
  'ROUND SNACK BOXES SET OF4

In [160]:
len(customer_sequence_Nov)

1378

In [154]:
for i in (customer_sequence_Nov):
    print(len(i))

1
1
2
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
3
1
1
2
1
1
1
1
2
2
1
2
1
5
3
2
2
1
1
1
2
1
2
1
2
1
2
1
1
1
1
1
1
1
1
1
2
1
1
1
1
3
1
2
1
1
1
1
1
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
2
1
1
2
1
1
1
1
4
3
1
1
1
1
1
1
2
3
1
1
1
1
4
2
1
1
1
1
2
1
1
35
1
1
2
1
1
1
1
2
1
1
1
1
2
3
1
2
1
1
1
1
4
1
2
1
2
1
2
1
1
7
1
1
2
1
1
2
1
1
2
4
1
1
8
1
1
1
1
2
1
1
1
2
3
2
2
4
1
1
1
1
1
2
1
1
1
1
2
4
1
1
10
1
1
5
2
1
3
1
1
2
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
2
1
2
2
1
2
3
1
1
1
1
1
2
1
1
1
1
2
1
2
4
1
1
1
1
3
1
1
1
1
2
2
1
1
1
1
1
1
1
3
1
1
1
1
1
1
1
1
1
1
1
1
6
1
3
3
2
1
1
1
1
2
1
1
1
2
1
1
2
1
1
2
1
1
2
1
1
1
1
1
1
1
1
3
1
1
2
1
1
1
1
2
1
2
1
1
3
1
1
1
6
1
1
1
2
1
1
1
1
1
1
1
1
2
1
2
1
3
2
1
1
2
1
1
1
1
1
1
1
1
1
1
2
1
4
2
1
2
1
1
3
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
2
1
1
1
2
1
1
1
2
1
1
1
1
1
2
1
1
1
1
2
1
2
1
1
3
2
1
4
1
1
3
1
1
2
1
1
2
1
2
1
1
1
1
2
1
2
2
1
1
1
1
1
3
1
1
3
2
2
1
1
1
1
2
2
1
1
1
1
1
3
1
1
2
1
1
1
1
2
4
1
1
2
1
1
3
1
1
1
2
2
1
1
3
2
1
2
1
1
1
1
1
1
1
1
1
1
1
3
1
1
1
1
1
1
1
1
1
1
1
5
1
3
1
3
3
1


In [155]:
customer_sequence_reducted_November_Supp5=list()
for i in (customer_sequence_Nov):
    if(len(i)>=5):
        customer_sequence_reducted_November_Supp5.append(i)

In [161]:
for i in (customer_sequence_reducted_November_Supp5):
    print(len(i))
print(len(customer_sequence_reducted_November_Supp5))

5
35
7
8
10
5
6
6
5
9
6
32
9
5
5
5
5
6
13
5
6
21


In [162]:
customer_sequence_reducted_November_Supp5[1]

[['DOORMAT NEW ENGLAND',
  'PANTRY SCRUBBING BRUSH',
  'COWBOYS AND INDIANS BIRTHDAY CARD ',
  'CARD SUKI BIRTHDAY',
  'SKULLS GREETING CARD',
  'SWALLOWS GREETING CARD',
  '12 PENCILS TALL TUBE SKULLS',
  'MIRRORED WALL ART SKULLS',
  'RED RETROSPOT LUGGAGE TAG',
  'POMPOM CURTAIN'],
 ['JUMBO BAG OWLS', 'JUMBO BAG PINK POLKADOT'],
 ['75 GREEN PETIT FOUR CASES',
  'BLACK/BLUE POLKADOT UMBRELLA',
  'EDWARDIAN PARASOL BLACK',
  'EDWARDIAN PARASOL NATURAL'],
 ['LUNCH BAG WOODLAND',
  'SET OF 4 PANTRY JELLY MOULDS',
  'LOVEBIRD HANGING DECORATION WHITE ',
  'HANGING ENGRAVED METAL HEART',
  'MODERN FLORAL STATIONERY SET',
  'WOOD STAMP SET FLOWERS',
  'PAPERWEIGHT VINTAGE COLLAGE',
  'PAPERWEIGHT KINGS CHOICE ',
  'GREEN JUICY FRUIT PHOTO FRAME',
  '3 HOOK PHOTO SHELF ANTIQUE WHITE',
  'S/4 VALENTINE DECOUPAGE HEART BOX',
  'BEWARE OF THE CAT METAL SIGN ',
  'AIRLINE LOUNGE,METAL SIGN',
  'HAND OVER THE CHOCOLATE   SIGN ',
  'CHOCOLATE THIS WAY METAL SIGN',
  'METAL SIGN EMPIRE TEA',
  'WA

In [158]:
outfile = open("customer_sequence_reducted_November_Supp5.pkl",'wb')

In [159]:
pickle.dump(customer_sequence_reducted_November_Supp5,outfile)
outfile.close()

In [128]:
from gsp import *

In [129]:
#signature requires: the dataset, the min support and the boolean verbose
result_set = apriori(customer_sequence_Nov, 5, verbose=True)

KeyboardInterrupt: 

In [139]:
import pickle


with open('customer_sequence_Nov_reducted_5', 'rb') as f:
    data = pickle.load(f)
len(data)

1378

In [140]:
data

[[['PARISIENNE CURIO CABINET',
   'SWEETHEART WALL TIDY ',
   'PINK  HEART SHAPE LOVE BUCKET ',
   'GINGHAM HEART  DOORSTOP RED',
   'RED HEART SHAPE LOVE BUCKET ',
   'FOOD CONTAINER SET 3 LOVE HEART ',
   'LARGE HEART MEASURING SPOONS',
   'DOORMAT HEARTS',
   'HANGING HEART JAR T-LIGHT HOLDER',
   'BROCANTE SHELF WITH HOOKS',
   'PLASTERS IN TIN VINTAGE PAISLEY ',
   'PANTRY MAGNETIC  SHOPPING LIST',
   'RECIPE BOX PANTRY YELLOW DESIGN',
   'SET OF 3 CAKE TINS PANTRY DESIGN ',
   'JAM MAKING SET WITH JARS',
   'SET OF 6 SPICE TINS PANTRY DESIGN',
   'PANTRY CHOPPING BOARD',
   'DOORMAT WELCOME TO OUR HOME',
   '16 PIECE CUTLERY SET PANTRY DESIGN',
   'SMALL WHITE RETROSPOT MUG IN BOX ',
   'BLACK/BLUE POLKADOT UMBRELLA',
   '20 DOLLY PEGS RETROSPOT',
   'SET/5 RED RETROSPOT LID GLASS BOWLS',
   'DOORMAT RED RETROSPOT',
   'SET/6 RED SPOTTY PAPER CUPS',
   'RED RETROSPOT SMALL MILK JUG',
   'RED RETROSPOT SUGAR JAM BOWL',
   'RETROSPOT LARGE MILK JUG',
   'SMALL RED RETROSPOT MUG IN 

In [80]:
data.sort(key=lambda tup: tup[1], reverse=True) 
data_combined1=list()
data_combined2=list()

for element in data:
    if(element[1]>=100):
        if(len(element[0]) == 1):
            data_combined1.append(element)
        elif(len(element[0]) >= 2):
            data_combined2.append(element)
        
            
print(len(data_combined1))
print(len(data_combined2))


len(data_combined2)

data_combined2.sort(key=lambda tup: tup[1], reverse=True) 
data_combined2

805
229


[([['WHITE HANGING HEART T-LIGHT HOLDER'],
   ['WHITE HANGING HEART T-LIGHT HOLDER']],
  291),
 ([['PARTY BUNTING'], ['PARTY BUNTING']], 218),
 ([['JUMBO BAG RED RETROSPOT'], ['JUMBO BAG RED RETROSPOT']], 217),
 ([['LUNCH BAG RED RETROSPOT'], ['LUNCH BAG RED RETROSPOT']], 213),
 ([['REGENCY CAKESTAND 3 TIER'], ['REGENCY CAKESTAND 3 TIER']], 184),
 ([['LUNCH BAG  BLACK SKULL.'], ['LUNCH BAG  BLACK SKULL.']], 179),
 ([['SET OF 3 CAKE TINS PANTRY DESIGN '],
   ['SET OF 3 CAKE TINS PANTRY DESIGN ']],
  178),
 ([['SPOTTY BUNTING'], ['SPOTTY BUNTING']], 168),
 ([['LUNCH BAG  BLACK SKULL.'], ['LUNCH BAG RED RETROSPOT']], 163),
 ([['LUNCH BAG RED RETROSPOT'], ['LUNCH BAG  BLACK SKULL.']], 163),
 ([['NATURAL SLATE HEART CHALKBOARD '], ['NATURAL SLATE HEART CHALKBOARD ']],
  161),
 ([['LUNCH BAG CARS BLUE'], ['LUNCH BAG CARS BLUE']], 159),
 ([['PARTY BUNTING'], ['SPOTTY BUNTING']], 158),
 ([['LUNCH BAG RED RETROSPOT'], ['LUNCH BAG PINK POLKADOT']], 155),
 ([['LUNCH BAG SPACEBOY DESIGN '], ['LUNC